## Imports


In [1]:
import torch
import torchvision
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter

## Dataset

In [2]:
IN_CHANNELS = 1
IMG_SIZE = 64
BATCH_SIZE = 128

In [3]:
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.5 for _ in range(IN_CHANNELS)], [0.5 for _ in range(IN_CHANNELS)])
])

In [4]:
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
loader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True)

## Model

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
def initialize_wieghts(model):

    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            torch.nn.init.normal_(m.weight.data ,0.0, 0.02)


In [7]:
class Generator(nn.Module):

    def __init__(self, img_channels):
        super().__init__()
        self.layers = nn.Sequential(
            self.gen_block(100, 1024, 4, 1, 0),
            self.gen_block(1024, 512, 4, 2, 1),
            self.gen_block(512, 256, 4, 2, 1),
            self.gen_block(256, 128, 4, 2, 1),
            nn.ConvTranspose2d(128, img_channels, 4, 2, 1),
            nn.Tanh()
        )

    def gen_block(self, in_channels, out_channels, kernel_size, stride, padding):
        block_layers = nn.Sequential(
            nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False
            ),
            nn.BatchNorm2d(out_channels),  
            nn.ReLU()
        )

        return block_layers
        
    def forward(self, x):
        x = self.layers(x)

        return x


In [8]:
class Discriminator(nn.Module):
    def __init__(self, img_channels):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Conv2d(img_channels, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2),
            self._block(64, 128, 4, 2, 1),
            self._block(128, 256, 4, 2, 1),
            self._block(256, 512, 4, 2, 1),
            nn.Conv2d(512, img_channels, kernel_size=4, stride=2, padding=0)
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,#
            ),

            nn.InstanceNorm2d(out_channels, affine=True),#
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.disc(x)


In [9]:
generator = Generator(IN_CHANNELS).to(device)
initialize_wieghts(generator)

critic = Discriminator(IN_CHANNELS).to(device)
initialize_wieghts(critic)


## Train

In [10]:
fixed_noise = torch.randn(32, 100, 1, 1).to(device)

In [11]:
EPOCHS = 15
LR = 0.00005
LAMBDA_GP = 10
n_critic = 5

optimizer_G = optim.Adam(generator.parameters(), lr=LR)

optimizer_C = optim.Adam(critic.parameters(), lr=LR)

writer_fake = SummaryWriter(f"logs_WGAN_PG/fake")
writer_real = SummaryWriter(f"logs_WGAN_PG/real")
step = 0

In [12]:
generator.train()

Generator(
  (layers): Sequential(
    (0): Sequential(
      (0): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
      (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): Sequential(
      (0): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (3): Sequential(
      (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (4): ConvTranspose2d(128, 

In [13]:
critic.train()

Discriminator(
  (disc): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (4): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (5): Conv2d(512, 1, kernel_size=(4, 4), stride=(2, 2))
  )
)

In [14]:
def gradient_penalty(critic, reals, fakes, device='cpu'):
    BATCH, C, H, W = reals.shape
    epsilone = torch.rand(BATCH, 1, 1, 1).repeat(1, C, H, W).to(device)

    interpolated_images = epsilone * reals + (1 - epsilone) * fakes
    critic_outputs = critic(interpolated_images)

    gradient = torch.autograd.grad(
        outputs=critic_outputs,
        inputs=interpolated_images,
        grad_outputs=torch.ones_like(critic_outputs),
        create_graph=True,
        retain_graph=True,
    )[0]

    gradient = gradient.view(gradient.shape[0], -1)
    gradient_norm = gradient.norm(2, dim=1)
    gradient_penalty = torch.mean((gradient_norm - 1) ** 2)
    
    return gradient_penalty 




In [15]:
for epoch in range(EPOCHS):

    for i, (real_images, _) in enumerate(loader):
        #Training Discriminator

        critic.train()

        for _ in range(0, n_critic):

            real_images = real_images.to(device)

            noise = torch.randn(real_images.shape[0], 100, 1, 1).to(device)
            fake_images = generator(noise)

            critic_real = critic(real_images).reshape(-1)
        
            critic_fake = critic(fake_images).reshape(-1)

            gp = gradient_penalty(critic, real_images, fake_images, device=device)
        
            loss_C = -(torch.mean(critic_real) - torch.mean(critic_fake)) + LAMBDA_GP * gp

            optimizer_C.zero_grad()
            loss_C.backward(retain_graph=True)
            optimizer_C.step()


        #Training Discriminator

        critic.eval()
        
        gen_fake = critic(fake_images).reshape(-1)

        loss_G = -torch.mean(gen_fake)

        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()


        if i== 0:
            print(
                f"Epoch [{epoch}/{EPOCHS}] Batch {i}/{len(loader)} \
                      Loss C: {loss_C:.4f}, loss G: {loss_G:.4f}"
            )

            with torch.no_grad():
                fake_images = generator(fixed_noise)
                data = real_images
                img_grid_fake = torchvision.utils.make_grid(fake_images[:32], normalize=True)
                img_grid_real = torchvision.utils.make_grid(data[:32], normalize=True)

                writer_fake.add_image(
                    "Mnist Fake Images", img_grid_fake, global_step=step
                )
                writer_real.add_image(
                    "Mnist Real Images", img_grid_real, global_step=step
                )
                step += 1

        

Epoch [0/15] Batch 0/469                       Loss C: 100.7502, loss G: -1.2579
Epoch [1/15] Batch 0/469                       Loss C: -53.6851, loss G: 56.1127
Epoch [2/15] Batch 0/469                       Loss C: -23.7363, loss G: 48.7729
Epoch [3/15] Batch 0/469                       Loss C: -15.9419, loss G: 50.2393
Epoch [4/15] Batch 0/469                       Loss C: -11.7129, loss G: 48.3039


KeyboardInterrupt: 